# ArcCi Training GUI

In [1]:
import os
#os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
#os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0
#import sys
#sys.path.append('/content/gdrive/S:/OSSP-master/CJ')
import tkinter as tk
from tkinter import *
# import tkinter.font as font
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import matplotlib.pyplot as plt
import numpy as np
# import argparse
from ctypes import *
from osgeo import gdal
from lib import utils # you have to upload to runtime session in google collab during every session
# import skimage
from skimage.segmentation import mark_boundaries
from lib import preprocess as pp
from lib.segment2 import segment_image2 as segment_image2
import lib.createCoco as CC
import uuid
# import imageio

#Global vars
MOUSE_X = 0
MOUSE_Y = 0
gauss_sigma0 = 3
feature_separation0 = 1
merging_cut = 30
classColors = ['#71b8eb', '#b1ddfc', '#707070', '#8d56ba', "#fff5a8", "#d97796"]

def hexToRGB(hexCode):
    return tuple(int(hexCode.lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
 
# Finds all the unique images from the given directory
def scrapeDir(src_dir):
    imageList = []

    for ext in utils.valid_extensions:   # valid_extensions is a list ['.tif','.tiff','.jpg']
        rawList = utils.get_image_paths(src_dir, keyword=ext) # utils.py module
        for rawImg in rawList:
            imageList.append(rawImg)

    # Save only the unique entries
    imageList = list(set(imageList))
    utils.remove_hidden(imageList)

    return imageList

#Colors
class colors:
    def __init__(self):
        self.lightMode = ['#e3e3fc', '#f7f7ff', 'black','yellow']
        self.darkMode = ['#7342bd','#000329','#6392ad','#bd0010']
        self.dunkMode = ["#3f3b40", "#203e45", "#e8e5c5", "#4a7c87"]

class colorPalette:
    def __init__(self):
        self.primaryColor = None
        self.secondaryColor = None
        self.tertiaryColor = None
        self.colorPalette = None
        self.initButton = None
    def setColorPalette(self, primaryColor, secondaryColor, tertiaryColor, initButton): #called during the config
        self.primaryColor = primaryColor #
        self.secondaryColor = secondaryColor # Side Frames
        self.tertiaryColor = tertiaryColor #
        self.fourthColor = initButton
    def configColors(self, palette):
        self.colorPalette = palette
        self.setColorPalette(self.colorPalette[0], self.colorPalette[1], self.colorPalette[2], self.colorPalette[3])
    
c = colors()
cP = colorPalette()
cP.configColors(c.dunkMode)

def errorDisplay(message, window):
    confirmWindow = tk.Toplevel()
    confirmWindow.config(bg=cP.secondaryColor)
    confirmWindow.resizable(False,False)
    
    confirmLabel = tk.Label(confirmWindow, text="ERROR: " + message, highlightthickness=0, fg=cP.tertiaryColor, bg=cP.secondaryColor)
    confirmLabel.grid(column=0, row=0, columnspan=2, pady=(10,10), padx=(10,10))
    
    returnButton = tk.Button(confirmWindow, text="Return", highlightthickness=0, fg=cP.tertiaryColor, bg=cP.secondaryColor ,command=lambda: [confirmWindow.destroy(), window.destroy()])
    returnButton.grid(column=1, row=1, pady=(10,20),padx=(0,50), sticky='nse')

# Buttons

In [2]:
class ButtonsLeft(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self.config(bg = cP.secondaryColor)
        self.parent = parent
        
        navLabel = tk.Label(self, text = "Navigation", fg = cP.tertiaryColor, bg = cP.primaryColor, pady = 20)
        navLabel.grid(column = 0, row = 0, pady = 0, sticky = 'nsew')
        
        yNavLabel = tk.Label(self, text = "Y", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor, width = 18, height = 0)
        yNavLabel.grid(column = 0, row = 1, sticky = 'nsew')
        
        # Slider that lets us traverse an image by its x and y coordinates
        self.yNavSlider = tk.Scale(self, from_ = 0, to_ = 0, length = 160, highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, orient = HORIZONTAL,
                             command = self.getYNavVal)
        self.yNavSlider.grid(column = 0, row = 2, pady = (0,6), sticky = 'ns')
        
        xNavLabel = tk.Label(self, text = "X", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor, width = 18, height = 0)
        xNavLabel.grid(column = 0, row = 3, sticky = 'nsew')
        
        # Instead of radio buttons we will attempt to use two sliders - one for each axis of the image
        self.xNavSlider = tk.Scale(self, from_ = 0, to_ = 0, length = 160, highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, orient = HORIZONTAL,
                             command = self.getXNavVal)
        self.xNavSlider.grid(column = 0, row = 4, pady = (0,4), sticky = 'ns')
        
        cocoLabel = tk.Label(self, text = "COCO Data", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor, width = 18, height = 2)
        cocoLabel.grid(column = 0, row = 5, pady = (5, 1), sticky = 'nsew')
        
        saveButton = tk.Button(self, text = "Save", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, width = 9, height = 2,
                        command = lambda: parent.event.save('save'))
        saveButton.grid(column = 0, row = 6, sticky = 'nsw') 
        
        saveAsButton = tk.Button(self, text = "Save As", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, width = 9, height = 2,
                           command = lambda: parent.event.save('saveAs'))
        saveAsButton.grid(column = 0, row = 6, sticky = 'nse') 

        self.tdsLabel = tk.Label(self, text='Current File Name:\n' + str(self.parent.data.COCOFileName), highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor)
        self.tdsLabel.grid(column = 0, row=7, pady=(5,5))
        
        nextSegButton = tk.Button(self, text = "Next Segment", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, width = 9, height = 2,
                           command = lambda: parent.event.nextSegment())
        nextSegButton.grid(column = 0, row = 10, sticky = 'nsew')
        
        cropButton = tk.Button(self, text = "Crop Image", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, width = 18, height = 2,
                           command = lambda: self.parent.imageDisplay.saveCroppedImage())
        cropButton.grid(column = 0, row = 11, pady = (20,1), sticky = 'nsew')
        
        
    def getYNavVal(self, newVal):
        self.yNav = newVal
        try:
            imageDisplay = self.parent.imageDisplay
            imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin, yNav = self.yNav)
            self.parent.data.resetSegments()
        except TypeError as error:
            pass
        except AttributeError as error:
            pass
    def getXNavVal(self, newVal):
        self.xNav = newVal
        try:
            imageDisplay = self.parent.imageDisplay
            imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin, xNav = self.xNav)
            self.parent.data.resetSegments()
        except TypeError as error:
            pass
        except AttributeError as error:
            pass

In [3]:
class ButtonsBottom(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self.config(bg = cP.primaryColor)
        self.parent = parent
        self.bind_all('<KeyPress>', self.keyPress)
  
        waterClassButton = tk.Button(self, text = "Water (1)", highlightthickness=0, bg = classColors[0], fg = 'black', width = 16, height = 2,
                                    command = lambda: parent.event.classify('water') )
        waterClassButton.grid(column = 1, row = 3, padx = (10,5), pady = (20,20), sticky = 'N')
        
        thinIceClassButton = tk.Button(self, text = "Thin Ice (2)", highlightthickness=0, bg = classColors[1], fg = 'black',width = 16, height = 2,
                                    command = lambda: parent.event.classify('thinIce') )
        thinIceClassButton.grid(column = 2, row = 3, padx = 5, pady = (20,20), sticky = 'N')
        
        shadowClassButton = tk.Button(self, text = "Shadow (3)", highlightthickness=0, bg = classColors[2], fg = 'black',width = 16, height = 2,
                                    command = lambda: parent.event.classify('shadow') )
        shadowClassButton.grid(column = 3, row = 3, padx = 5, pady = (20,0), sticky = 'N')
        
        #!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ADD FUNCTIONALITY TO THIS BUTTON -BROKEN
        subIceClassButton = tk.Button(self, text = "Submerged Ice (4)", highlightthickness=0, bg = classColors[3], fg = 'black',width = 16, height = 2,
                                    command = lambda: parent.event.classify('subIce') )
        subIceClassButton.grid(column = 4, row = 3, padx = 5, pady = (20,20), sticky = 'N')
        
        snowClassButton = tk.Button(self, text = "Ice/Snow (5)", highlightthickness=0, bg = classColors[4], fg = 'black',width = 16, height = 2,
                                    command = lambda: parent.event.classify('snow') )
        snowClassButton.grid(column = 5, row = 3, padx = 5, pady = (20,20), sticky = 'N')
        
        meltPondClassButton = tk.Button(self, text = "Melt Pond (6)", highlightthickness=0, bg = classColors[5], fg = 'black',width = 16, height = 2,
                                    command = lambda: parent.event.classify('meltPond') )
        meltPondClassButton.grid(column = 6, row = 3, padx = 5, pady = (20,20), sticky = 'N')

        self.imgIDLabel = tk.Label(self, text='Current Image ID: ' + str(self.parent.data.imgID), highlightthickness=0, bg = cP.primaryColor, fg = 'white')
        self.imgIDLabel.grid(column=1,row=4, pady=(0, 10), columnspan=6)
        
    def keyPress(self, e):
        try:
            if str(e.char) == '1':
                self.parent.event.classify('water')
            elif str(e.char) == '2':
                self.parent.event.classify('thinIce')
            elif str(e.char) == '3':
                self.parent.event.classify('shadow')
            elif str(e.char) == '4':
                self.parent.event.classify('subIce')
            elif str(e.char) == '5':
                self.parent.event.classify('snow')
            elif str(e.char) == '6':
                self.parent.event.classify('meltPond')
        except Exception as e:
            pass 

In [4]:
class ButtonsRight(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self.config(bg = cP.secondaryColor)
        self.parent = parent
        self.zoom = 50
        self.stringVar = tk.StringVar(self, "pan")
        self.gaussInput = tk.DoubleVar(self, 3.0)
        self.separationInput = tk.DoubleVar(self, 1.0)
        
        zoomLabel = tk.Label(self, text = "Zoom +-", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, pady = 20)
        zoomLabel.grid(column = 0, row = 0, pady = 0, sticky = 'nsew')
        
        #Zoom Slider: allows us to zoom in on an image
        self.zoomSlider = tk.Scale(self, from_ = 50, to = 400, length = 160, highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, orient = HORIZONTAL, 
                              relief = RAISED, command = self.getZoomSliderVal)
        self.zoomSlider.grid(column = 0, row = 1, pady = 0, sticky = 'nsew')
        
        self.zoomSlider.set(100)
        
        autoButton = tk.Button(self, text = "Auto", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, width=20, height=1,
                                    command = lambda: parent.event.imageSizeLock())    
        autoButton.grid(column = 0, row = 2, sticky = 'nsew')
        
        imageTypeLabel = tk.Label(self, text = "Image type:", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor)
        imageTypeLabel.grid(column = 0, row = 3, pady = (5, 5), sticky = 'nsew')
        
        wvmRadio = tk.Radiobutton(self, value = "wv02_ms", variable = self.stringVar, command = self.changeImageType, selectcolor = cP.primaryColor, text = "wv02_ms", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor)
        wvmRadio.grid(column = 0, row = 4, padx = (0, 0), sticky = "nsw")

        panRadio = tk.Radiobutton(self, value = "pan", variable = self.stringVar, command = self.changeImageType, selectcolor = cP.primaryColor, text = "pan", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor)
        panRadio.grid(column = 0, row = 4, padx = (120, 0), sticky = "nsw")
        
        rgbRadio = tk.Radiobutton(self, value = "srgb", variable = self.stringVar, command = self.changeImageType, selectcolor = cP.primaryColor, text = "srgb", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor)
        rgbRadio.grid(column = 0, row = 4, padx = (75, 0), sticky = "nsw")
        
        # Initialize Image Button
        initImageButton = tk.Button(self, text = "Initialize Image", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, width = 18, height = 2,
                                   command = lambda: parent.event.initializeImage())
        #initImageButton['font'] = font1
        initImageButton.grid(column = 0, row = 5, pady = (20,1), sticky = 'nsew')
        
        nextImageButton = tk.Button(self, text = ">", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, width = 9, height = 2,
                                   command = lambda: parent.event.nextImage())
        nextImageButton.grid(column = 0, row = 6, sticky = 'nse', pady = (0, 20))
        
        prevImageButton = tk.Button(self, text = "<",  highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, width = 9, height = 2,
                                   command = lambda: parent.event.prevImage())
        prevImageButton.grid(column = 0, row = 6, sticky = 'nsw', pady = (0, 20))
        
        gaussLabel = tk.Label(self, text = "Gauss Sigma", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor)
        gaussLabel.grid(column = 0, row = 7, pady = (0, 2), sticky = 'nsew')
        
        self.gaussSigmaSlider = tk.Scale(self, from_ = 1, to_ = 10, length = 120, highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, 
                                         orient = HORIZONTAL, relief = RAISED,command = self.gaussSigmaChange, resolution=0.1)
        self.gaussSigmaSlider.grid(column = 0, row = 8, padx = (3, 0), pady = (0,10), sticky = 'nse')
        self.gaussSigmaSlider.set(3)
        
        gaussSigmaInput = tk.Entry(self, width=6, fg = cP.tertiaryColor, bg = cP.primaryColor, textvariable=self.gaussInput, justify='center')
        gaussSigmaInput.bind("<Return>", self.gaussText)
        gaussSigmaInput.grid(column = 0, row = 8, padx = (3, 0), pady = (0,10), sticky = 'nsw')
        
        featureLabel = tk.Label(self, text = "Feature Separation", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor)
        featureLabel.grid(column = 0, row = 9, pady = (0,2), sticky = 'nsew')
        
        self.featSeparateSlider = tk.Scale(self, from_ = 1, to_ = 30, length = 120, highlightthickness=0, fg = cP.tertiaryColor, bg = cP.primaryColor, 
                                           orient = HORIZONTAL, relief = RAISED,command = self.featureSeparationChange)
        self.featSeparateSlider.grid(column = 0, row = 10, padx = (3, 0), pady = (0,10), sticky = 'nse')
        self.featSeparateSlider.set(1)
        
        featureSeparationInput = tk.Entry(self, width=6, fg = cP.tertiaryColor, bg = cP.primaryColor, textvariable=self.separationInput, justify='center')
        featureSeparationInput.bind("<Return>", self.featureText)
        featureSeparationInput.grid(column = 0, row = 10, padx = (3, 0), pady = (0,10), sticky = 'nsw')

        self.segLabel = tk.Label(self, text = "Current Segment: ", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor, width = 18, height = 2)
        self.segLabel.grid(column = 0, row = 11, pady = (1, 1), sticky = 'nsew') 
        
        self.leftLabel = tk.Label(self, text = "Segments Left: ", highlightthickness=0, fg = cP.tertiaryColor, bg = cP.secondaryColor, width = 18, height = 2)
        self.leftLabel.grid(column = 0, row = 12, pady = (1, 1), sticky = 'nsew') 
    
    
    def changeImageType(self):
        self.parent.data.imgType = self.stringVar.get()
        
    def gaussText(self, newVal):
        global gauss_sigma0
        
        try: 
            self.parent.data.deleteImgData()
            gauss_sigma0 = min(max(float(self.gaussInput.get()), 1), 10)
            self.gaussSigmaSlider.set(gauss_sigma0)
            self.gaussInput.set(gauss_sigma0)
            try:
                imageDisplay = self.parent.imageDisplay
                imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin)
                self.parent.data.resetSegments()
            except TypeError as error:
                pass
            except AttributeError as error:
                pass
        except Exception as error:
            pass
        
    def featureText(self, newVal):
        global gauss_sigma0
        
        try:
            self.parent.data.deleteImgData()
            feature_separation0 = min(max(float(self.separationInput.get()), 1), 30)
            self.featSeparateSlider.set(feature_separation0)
            self.separationInput.set(feature_separation0)
            try:
                imageDisplay = self.parent.imageDisplay
                imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin)
                self.parent.data.resetSegments()
            except TypeError as error:
                pass
            except AttributeError as error:
                pass
        except Exception as error:
            pass
        
    def gaussSigmaChange(self, newVal):
        global gauss_sigma0
        
        self.parent.data.deleteImgData()
        gauss_sigma0 = float(newVal)
        self.gaussInput.set(gauss_sigma0)
        try:
            imageDisplay = self.parent.imageDisplay
            imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin)
            self.parent.data.resetSegments()
        except TypeError as error:
            pass
        except AttributeError as error:
            pass
        
    def featureSeparationChange(self, newVal):
        global feature_separation0
        
        self.parent.data.deleteImgData()
        feature_separation0 = int(newVal)
        self.separationInput.set(feature_separation0)
        try:
            imageDisplay = self.parent.imageDisplay
            imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin)
            self.parent.data.resetSegments()
        except TypeError as error:
            pass
        except AttributeError as error:
            pass
    
    def getZoomSliderVal(self, newVal):
        self.zoom = newVal
        try:
            imageDisplay = self.parent.imageDisplay
            imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin)
            self.parent.data.resetSegments()
        except TypeError as error:
            pass
        except AttributeError as error:
            pass

# Image Display

In [5]:
class ImageDisplay(tk.Frame):
    def __init__(self, parent):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        
        self.displayImage = None
        self.cropImage = None
        self.imageZoom = None
        self.segmentedImage = None
    
        self.dispXDim , self.dispYDim, = 0, 0
        self.zoomWinX, self.zoomWinY = 0, 0
        
        self.prevZoom = 0

        #Image Canvas
        self.graph = plt.figure(figsize = [8, 4]) # width x height
        self.graph.subplots_adjust(left = 0.01, right = 0.99,
                                 bottom = 0.05, top = 0.99,
                                 wspace = 0.01, hspace = 0.01)
        imageCanvas = FigureCanvasTkAgg(self.graph, self)
        #self is the Tk canvas that we are adding self.graph to
        imageCanvas.draw()
        imageCanvas.get_tk_widget().grid(column = 2, row = 2) #Plot the canvas as a widget
        #!TODO: add functionalilty with the event manager
        
        self.homescreen()

    # Function contatining the homescreen
    def homescreen(self): #The screen the user will be met with upon opening the GUI
        plt.clf()
        openingMsg = "Welcome to ArcCI semantic segmentation training \n Press <Initialize Image> to begin."
        
        self.graph.set_facecolor(cP.fourthColor)
        
        openingScreen = self.graph.add_subplot(1, 1, 1)
        openingScreen.text(.5, .55, openingMsg, horizontalalignment = 'center', verticalalignment = 'top', weight = 'bold')
        openingScreen.axis('off')
        
        self.graph.canvas.draw()  
        
    # Function called when user presses 'initialize image' button
    def initializeImage(self):
        try:
            if self.parent.data.imgType == 'wv02_ms' or self.parent.data.imgType == 'pan':
                self.displayImage = utils.create_composite([self.parent.data.originalImage[0, :, :],
                                                             self.parent.data.originalImage[0, :, :],
                                                             self.parent.data.originalImage[0, :, :]],
                                                             dtype=np.uint8)

            elif self.parent.data.imgType == 'srgb':
                self.displayImage = utils.create_composite([self.parent.data.originalImage[0, :, :],
                                                             self.parent.data.originalImage[1, :, :],
                                                             self.parent.data.originalImage[2, :, :]],
                                                             dtype=np.uint8)
        except IndexError:
            self.parent.event.nextImage()
                        
        self.dispXDim, self.dispYDim = np.shape(self.displayImage)[0:2]

    # Function containing the loading screen
    def loadingScreen(self):
        #This function displays a loading screen between uploading images
        plt.clf()
        loadingText = "Images are loading, please wait..."
        self.graph.set_facecolor(cP.fourthColor)
        ax = self.graph.add_subplot(1, 1, 1, adjustable = 'datalim', frame_on = False)
        ax.text(0.5, 0.5, loadingText, horizontalalignment = 'center', verticalalignment = 'center')
        ax.axis('off')
        
        self.graph.canvas.draw()
    
    # Function called when user presses crop image button
    def saveCroppedImage(self):
        # Generates 32 char (36 with 4 -) ID for image to be 'tagged' with; for mask and image matching
        imgID = str(uuid.uuid1())
        newFileName = os.path.relpath(self.parent.data.cropDir + '/img-' + imgID + '.jpg')
        # Makes it a name ^^
        # Saves image
        plt.imsave(newFileName, self.imageData)
        
    # Function to generate an image mask of segments
    def maskImage(self, image):
        global classColors
        
        for s, classedSegment in enumerate(self.parent.data.labelVector):
            classifiedID = classedSegment[1]
            unSegmentArea = self.segmentedImage != classifiedID
            segmentArea = self.segmentedImage == classifiedID
            empty = np.zeros(image[:, :, 0].shape)
            otherEmpty = np.zeros(image[:, :, 0].shape)
            empty.fill(1)
            empty *= unSegmentArea
            
            for c, color in enumerate(hexToRGB(classColors[classedSegment[0]])):
                otherEmpty.fill(color/255)
                otherEmpty *= segmentArea
                image = image.astype(np.float64)
                image[:, :, c] *= empty + otherEmpty
                image = image.astype(np.uint8)
                
        return image
          
    # Update function called every cycle to update the image data displayed
    def updateImages(self, x, y, **kwargs):
        global gauss_sigma0, feature_separation0, merging_cut
        
        xNav = kwargs.get("xNav", None)
        yNav = kwargs.get("yNav", None)
        
        plt.clf()
        
        if self.parent.event.imageLocked == True:
            zoomSize = 128
            self.parent.rButtons.zoomSlider.set(128)
        elif self.parent.event.imageLocked == False:
            zoomSize = int(self.parent.rButtons.zoom)
        else:
            zoomSize = int(self.parent.rButtons.zoom)
            
        zoomSize = min(min(zoomSize, self.dispXDim/2), self.dispYDim/2)

        self.imageZoom = zoomSize
        
            
        
        # Sets color of image cover when it reaches 256x256
        if zoomSize == 128:
            color = 'blue'
        elif zoomSize <= 132 and zoomSize > 128:
            color = 'green'
        elif zoomSize >= 124 and zoomSize < 128:
            color = 'green'
        else:
            color = 'red'
        
        xMin = y                     # xMin: row, is onclick ydata 
        xMax = y + (zoomSize * 2)    # xMax: row, is onclick ydata
        yMin = x                     # yMin: col, is onclick xdata 
        yMax = x + (zoomSize * 2)    # yMax: col, is onclick xdata 
        
        self.xMin = xMin
        self.xMax = xMax
        self.yMin = yMin
        self.yMax = yMax

        # Store the zoom window corner coordinates for reference in onclick()
        # xMin and yMin are defined backwards
        if yNav is not None:
            self.zoomWinX = yMin + int(yNav)
        if xNav is not None:
            self.zoomWinY = xMin + int(xNav)
        #for nav slider ^
        
        xMin += self.zoomWinX
        xMax += self.zoomWinX
        yMin += self.zoomWinY
        yMax += self.zoomWinY
        
        if xMin < 0:
            xMin = 0
            xMax = zoomSize*2 - 1
        if xMax > self.dispXDim:
            xMax = self.dispXDim
            xMin = xMax - (zoomSize*2 - 1)
        if yMin < 0:
            yMin = 0
            yMax = zoomSize*2 - 1
        if yMax > self.dispYDim:
            yMax = self.dispYDim
            yMin = yMax - (zoomSize*2 - 1)
            
        if xMin == xMax:
            xMax += 1
        if yMin == yMax:
            yMax += 1
        
        if self.displayImage is not None:
            self.imageHeight = self.displayImage.shape[0]
            self.imageWidth = self.displayImage.shape[1]
            if zoomSize != self.prevZoom:
                self.parent.event.configNavSliders(zoom = zoomSize)
                self.prevZoom = zoomSize
        
        croppedImage = self.displayImage[int(xMin):int(xMax), int(yMin):int(yMax)]

        imageData = croppedImage
        self.imageData = croppedImage
        
        self.imageData = imageData
        
        if self.parent.data.imgType == 'pan':
            imageData=croppedImage[:,:,0]
            imageData=np.reshape(imageData, (1, imageData.shape[0], imageData.shape[1]))
        
        if self.parent.data.imgType == 'srgb':
            imageData=np.stack( (imageData[:,:,0], imageData[:,:,1], imageData[:,:,2]), axis=0)  #(row, col, 3)-> (3, row, col)
        
        watershedImage = segment_image2(croppedImage, gauss_sigma0, feature_separation0, 
                                  image_type=self.parent.data.imgType, merging_cut = merging_cut)
        
        self.segmentedImage = np.ndarray.astype(watershedImage, c_uint32)
        self.parent.data.segmentedImage = self.segmentedImage
            
        if np.max(watershedImage) > 1:
            watershedResult = mark_boundaries(croppedImage, watershedImage)  
            watershedResult = 255 * watershedResult
            watershedResult = watershedResult.astype(np.uint8) 

        else:
            watershedResult = np.zeros(np.shape(watershedImage)).astype(np.uint8)

        # Left side image axis
        displayAxis = self.graph.add_subplot(1, 2, 1)
        displayAxis.imshow(self.displayImage, interpolation='None', vmin=0, vmax=255)
        displayAxis.axvspan(yMin,
                   yMax,
                   1. - float(xMax) / self.dispXDim,
                   1. - float(xMin) / self.dispXDim,
                   color=color,
                   alpha=0.3) 
        displayAxis.set_xlim([0, np.shape(self.displayImage)[1]])
        displayAxis.tick_params(axis='both',  # changes apply to the x-axis
                       which='both',  # both major and minor ticks are affected
                       bottom=False,  # ticks along the bottom edge are off
                       top=False,  # ticks along the top edge are off
                       left=False,
                       right=False,
                       labelleft=False,
                       labelbottom=False)
        #displayAxis.set(facecolor = "pink")
        displayAxis.set_label('displayAxis')
            
        # Image 2 (Zoomed in image, with segment highlight)
        colorImage = watershedResult
        
        currentSegments = self.parent.data.getCurrentSegment()

        self.parent.data.findImgId()
        
        self.parent.rButtons.leftLabel.config(text = "Segments Left: " + str(len(self.parent.data.segmentList) - len(self.parent.data.labelVector)))
        self.parent.lButtons.tdsLabel.config(text='Current File Name:\n' + str(self.parent.data.COCOFileName))
        self.parent.classificationButtons.imgIDLabel.config(text='Current Image ID: ' + str(self.parent.data.imgID))
        
        segmentID = currentSegments
        
        for i in segmentID:
            try: 
                segmentArea = np.logical_or(segmentArea, (self.segmentedImage == i))
            except NameError as e:
                segmentArea = (self.segmentedImage == i)
                
        unSegmentArea = np.bitwise_not(segmentArea)
            
        empty = np.zeros(colorImage[:, :, 0].shape)
        empty.fill(1)
        otherEmpty = np.zeros(colorImage[:, :, 0].shape)
        otherEmpty.fill(1)
        
        empty *= unSegmentArea
        otherEmpty *= segmentArea
        
        empty *= 0.5
        
        colorImage = colorImage.astype(np.float64)
        
        colorImage[:, :, 0] *= empty + otherEmpty
        colorImage[:, :, 1] *= empty + otherEmpty
        colorImage[:, :, 2] *= empty + otherEmpty
        
        colorImage = colorImage.astype(np.uint8)
        
        colorImage = self.maskImage(colorImage)
        
        # Right side image display axis
        segmentAxis = self.graph.add_subplot(1, 2, 2)
        segmentAxis.imshow(colorImage, interpolation='None', vmin=0, vmax=255)
        segmentAxis.tick_params(axis='both',  # changes apply to the x-axis
                      which='both',  # both major and minor ticks are affected
                       bottom=False,  # ticks along the bottom edge are off
                       top=False,  # ticks along the top edge are off
                       left=False,
                       right=False,
                       labelleft=False,
                       labelbottom=False)
        segmentAxis.set_label('segmentAxis')

        self.graph.canvas.mpl_connect('button_press_event', self.parent.event.onClick)
        # Updating the plots
        self.graph.canvas.draw()

# Training Window

In [6]:
class TrainingWindow(tk.Frame):
    def __init__(self, parent, imgList, imgType, rasterList, activate_autorun = False):#!, img_list, username, im_type, activate_autorun=False):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.parent.title("ArcCI Deep Learning Training GUI")
        self.parent.config(bg = classColors[3])
        # self.parent.tk.call('wm', 'iconphoto', self.parent._w, tk.PhotoImage(file='Iceberg.png'))

        #!TODO create custom icon: self.parent.iconbitmap('vectorstock_7199163_icon.ico') # Designed by freepik (Image #7199163 at VectorStock.com)
        
        # the bottom and right buttons are declared before the left buttons
        # the left buttons require that we augment the nav sliders to match the size of any given image using gdal metadata.
    
        #Data manager object
        self.data = DataManager(imgList, imgType, rasterList, self)
        
        self.classificationButtons = ButtonsBottom(self)
        self.classificationButtons.grid(column = 1, row = 1, columnspan = 3, sticky="NEW")
        
        self.rButtons = ButtonsRight(self)
        self.rButtons.grid(column = 2, row = 0, rowspan = 3, sticky="NSEW")

        self.lButtons = ButtonsLeft(self)
        self.lButtons.grid(column = 0, row = 0, rowspan = 3, sticky="NSEW") 
    
        # Image display window
        self.imageDisplay = ImageDisplay(self)
        self.imageDisplay.grid(column = 1, row = 0)
        
        # Manager for all the GUI events (e.g. button presses)
        self.event = EventManager(self)    # Define event_manager functions, should be found in EventManager()
        
        
        #self.progress_bar = ProgressBar(self)
        #self.progress_bar.grid(column=1, row=0)   # Progress info shown in col=1, row=0 (top right corner)

        #self.progress_bar.update_progress()
        
        #!TODO: add live prediction object

    def exitGui(self):
        self.parent.quit()
        self.parent.destroy()

# Data Manager

In [7]:
class DataManager:
    def __init__(self, availableImages, imgType, rasterList, parent):
        # Image and segment data (populated in load_image())
        self.originalImage = None
        self.segmentedImage = None
        
        self.parent = parent

        # Variable Values   (populated in load_training_data())
        self.labelVector = []
        self.segmentList = []
        self.featureMatrix = []
        self.segNumber = [-1]

        self.imgID = None
        
        self.tracker = 0   # Number of segment sets added from the current image
        self.imageIndex = 0   # Index for progressing through available images
        
        # COCO Converter
        self.CC = CC.createCOCO()
        
        #For GeoJSON conversion - populated in loadImage()
        self.prevImageData = None

        self.newFileName = None
        self.COCOFileName = None
        
        # Global Static Values
        self.imgType = imgType
        self.availableImages = availableImages
        self.rasterList = rasterList
        
        # Image Static Value (populated in load_image())
        self.wb_ref = None
        self.br_ref = None
        self.imgDate = None
        self.imageName = None
        
        # List of output directories that are required for the program to function properly
        outputDir = os.path.relpath('WriteGUI')
        self.tdsDir = os.path.relpath(outputDir + '/TDSCOCO')
        self.cropDir = os.path.relpath(outputDir + '/CroppedImages/')
        self.maskDir = os.path.relpath(outputDir + '/MaskedImages')
        requiredDirs = [outputDir, self.tdsDir, self.cropDir, self.maskDir]
        # Checks if required directories exist, and makes them if they don't
        for dirs in requiredDirs:
            if os.path.isdir(dirs) == False:
                os.mkdir(dirs)
        
    # Function that returns the imgWidthList, and imgHeightList
    def getSizeLists(self):
        imgWidthList = []
        imgHeightList = []
        i = 0 # indexing var
        while i < len(self.rasterList):
            imgWidthList.append(self.rasterList[i].RasterXSize)
            imgHeightList.append(self.rasterList[i].RasterYSize)
            i += 1
        return imgWidthList, imgHeightList
    
    # Function to load next image
    def loadNextImage(self):
        # Increment the image index
        self.imageIndex += 1
        # Loop im_index based on the available number of images
        self.imageIndex = self.imageIndex % len(self.availableImages)
        # Load the new data
        self.loadImage()

    # Function to load previous image
    def loadPreviousImage(self):
        # Increment the image index
        self.imageIndex -= 1
        # Loop im_index based on the available number of images
        self.imageIndex = self.imageIndex % len(self.availableImages)
        # Load the new data
        self.loadImage()

    # Function to load image data for display
    def loadImage(self):
        # Loads the optical and segmented image data from disk. Should only be called from
        #   loadNextImage an loadPreviousImage
        fullImageName = self.availableImages[self.imageIndex]

        self.imageName = os.path.splitext(os.path.split(fullImageName)[1])[0]

        srcDS = gdal.Open(fullImageName, gdal.GA_ReadOnly)

        # Read the image date from the metadata
        metadata = srcDS.GetMetadata()
        self.imgDate = pp.parse_metadata(metadata, self.imgType)

        # Determine the datatype
        src_dtype = gdal.GetDataTypeSize(srcDS.GetRasterBand(1).DataType)

        # Calculate the reference points from the image histogram
        lower, upper, wb_ref, br_ref = pp.histogram_threshold(srcDS, src_dtype)
        self.wb_ref = np.array(wb_ref, dtype=c_uint8)
        self.br_ref = np.array(br_ref, dtype=c_uint8)
        
        # Load the image data
        imageData = srcDS.ReadAsArray()     # <class 'numpy.ndarray'> uint8 (3, 7471, 8242)
        if imageData.ndim == 2:
            imageData = imageData.reshape((1,imageData.shape[0],imageData.shape[1]))
         # Close the GDAL dataset
        srcDS = None

        # Rescale the input dataset using a histogram stretch
        imageData = pp.rescale_band(imageData, lower, upper)
        # Apply a white balance to the image
        imageData = pp.white_balance(imageData, self.wb_ref.astype(np.float64), float(np.amax(self.wb_ref)))
        # Convert the input data to c_uint8
        self.originalImage = np.ndarray.astype(imageData, c_uint8) # Necessary
        self.prevImageData = self.originalImage
        
        imageData = None
        watershedImage = None
           
    # Function to return the length of labelVector
    def getNumLabels(self):
        return len(self.labelVector)
    
    # Function that appends [label, segmentID] to labelVector
    def appendLabel(self, label, segmentID):
        self.tracker += 1
        self.labelVector.append([label, segmentID])
        
    # Removes the last value from labelVector
    def removePrevLabel(self):
        self.labelVector.pop()
        self.tracker -= 1
        
    # Returns the length of segmentList
    def getNumSegments(self):
        return len(self.segmentList)
    
    # Returns the currently highlighted segment
    def getCurrentSegment(self):
        if len(self.segmentList) < 1:
            self.addSegments()
        return [self.segmentList[i] for i in self.segNumber]
    
    # Adds one extra segment
    def addSingleSegment(self, newSegment):
        self.segmentList.append(newSegment)
        
    # Is used to trim segmentList to the length of segmentList
    def trimSegmentList(self):
        self.segmentList = self.segmentList[:len(self.labelVector)]
        
    # Resets all segment variables to their initialized values, and then generates more segments
    def resetSegments(self):
        if self.parent.data.segmentedImage is not None:
            self.segmentList = []
            self.labelVector = []
            self.segNumber = [-1]
            self.addSegments()
        
    # Adds segments to a list to be classified
    def addSegments(self):
        segmentedImage = self.parent.imageDisplay.segmentedImage
        
        if len(self.segmentList) == 0:
            for n, segment in enumerate(np.unique(segmentedImage)):
                self.segmentList.append(segment.item())
            
        if len(self.labelVector) > 0:
            self.segNumber = [len(self.segmentList) - (len(self.labelVector))]
        else:
            self.segNumber = [-1]

    # Function used to automatically delete data when user changes values of gauss sigma and/or feature separation
    def deleteImgData(self):
        if self.COCOFileName != None:
            masterDict = self.CC.fromCOCO(self.COCOFileName)

            # Loop checking for images with the right ID to be deleted and deleting them
            for i in masterDict['images']:
                if i['id'] == self.imgID:
                    masterDict['images'].remove(i)

            # Loop to remove all segment data with matching imgID
            for i in reversed(masterDict['annotation']):
                if i['image_id'] == self.imgID:
                    masterDict['annotation'].remove(i)

            # Save changes to the dataset
            self.CC.toJSON(self.COCOFileName, masterDict)
        
        
    # Creates file name and path for COCO dataset
    def createCOCOFileName(self, saveType):
        if saveType == 'save':
            self.COCOFileName = os.path.relpath(self.tdsDir + '/COCOTDS.json') # concatenate target directory with file index
        # If user presses save as button
        elif saveType == 'saveAs':
            files = [('JSON Object', '*.json')]
            self.COCOFileName = os.path.relpath(str(tk.filedialog.asksaveasfilename(filetypes = files, defaultextension = files, initialdir = self.tdsDir)))
            
    # Generates an ID for images to be tagged with
    def findImgId(self):
        self.imgID = self.imageName[-36:] # Assigns last 36 characters (UUID) of img name to imgID

# Event Manager

In [8]:
class EventManager:
    def __init__(self, parent):
        self.i = 0
        self.parent = parent
        self.isActive = False #Keeps events from happening when images are loading.
        #self.beacon = False # beacon that requests reconfig - necessary?
        self.gate = True # Gate that allows us to initialize an index variable for navigating images
        self.imgWidthList, self.imgHeightList = self.parent.data.getSizeLists() # Create size lists for nav config method
        self.countList = []
        self.CC = CC.createCOCO()
        self.imageLocked = False
        
    def activate(self):
        self.isActive = True
        
    def deactivate(self):
        self.isActive = False
        
    # Cycles to next highlighted segment
    def nextSegment(self):
        if self.parent.data.getNumLabels() == self.parent.data.getNumSegments():
                self.parent.data.addSegments()
        
        classified = True
        
        self.parent.data.segNumber = [-1]
        startSegment = self.parent.data.segNumber[0]
        
        while classified:
            self.parent.data.segNumber[0] += 1
        
            if self.parent.data.segNumber[0] >= len(self.parent.data.segmentList):
                self.parent.data.segNumber[0] -= len(self.parent.data.segmentList)
                
            segmentID = self.parent.data.getCurrentSegment()
            equalsSegment = list(segmentID == label[1] for label in self.parent.data.labelVector)
            classified = True in equalsSegment
            
            if self.parent.data.segNumber == startSegment:
                classified = False

        if not self.isActive:
            return
        
        imageDisplay = self.parent.imageDisplay
        imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin)
        

    # Is used by the 'auto' button to lock the image at size 256x256px
    def imageSizeLock(self):
        if not self.isActive:
            return

        if self.imageLocked == False:
            self.imageLocked = True
        elif self.imageLocked == True:
            self.imageLocked = False

        imageDisplay = self.parent.imageDisplay
        imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin)

    # Function to track onClick events for the gui, currently is used to select segments by clicking on them
    def onClick(self, event):
        global MOUSE_X, MOUSE_Y
        if not self.isActive:
            return
        if event.inaxes is not None:
            # Local Vars
            axesProperties = event.inaxes.properties()
            x, y = 0, 0
            
            # If the click was in the segmented image, or 'colorImg', display plot; Is used to select the segment a user clicks on as the active segment [NOT WORKING]
            if axesProperties['label'] == 'segmentAxis':
                winX = int(event.xdata)
                winY = int(event.ydata)
                x = winX
                y = winY

                segID = self.parent.imageDisplay.segmentedImage[y, x]
                
                # Iterate through list of segments to find matching segment info according to segID
                endI = None
                for i in self.parent.data.segmentList:
                    if i == segID: # Compare segID with current iterations segmentID to determine which iteration contains the matching segmentID
                        endI = i

                endS = None
                for s, segment in enumerate(self.parent.data.segmentList):
                    if segID == segment:
                        endS = s
                
                if str(event.button) == "MouseButton.LEFT":
                    if str(event.key) != "control":
                        self.parent.imageDisplay.currentSegment = [endI]
                        self.parent.data.segNumber = [endS]
                    else:
                        try:
                            self.parent.imageDisplay.currentSegment.append(endI)
                            self.parent.data.segNumber.append(endS)
                        except AttributeError as error:
                            self.parent.data.segNumber.append(endS)
                            self.parent.imageDisplay.currentSegment = self.parent.data.getCurrentSegment().append(endI)

                imageDisplay = self.parent.imageDisplay
                
                # Update Images to reflect the selected segment change
                imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin)

                # MOUSE_X = x
                # MOUSE_Y = y


            # If click was in left image window it will move the top left of the view to the mouse cursor 
            if axesProperties['label'] == 'displayAxis': 
                imageDisplay = self.parent.imageDisplay
                
                x = int(event.xdata) - imageDisplay.zoomWinY
                y = int(event.ydata) - imageDisplay.zoomWinX
                
                self.parent.lButtons.xNavSlider.set(int(event.xdata))
                self.parent.lButtons.yNavSlider.set(int(event.ydata))
                
                self.parent.data.resetSegments()
                
                imageDisplay.updateImages(imageDisplay.yMin, imageDisplay.xMin)
                
    
    # Function to save all current data that needs saved, is called a lot
    def save(self, saveType="save"):
        
        if self.parent.data.labelVector == []:
            return
        print("Saving Data...")
        
        segmentedImage = self.parent.imageDisplay.segmentedImage
        
        segmentList = self.parent.data.segmentList
        labelVector = self.parent.data.labelVector
        
        countList = []

        for s, segment in enumerate(segmentList):
            classificationNumber = None
            for l, label in enumerate(labelVector):
                if label[1] == segment:
                    classificationNumber = label[0]
            segmentArray = segmentedImage == segment
            flatList = segmentArray.flatten()
            currentSeg = False
            segTotal = 0
            counts = []
            for i, index in enumerate(flatList):
                if currentSeg != flatList[i]:
                    currentSeg = flatList[i]
                    counts.append(segTotal)
                    segTotal = 0
                segTotal += 1
            countList.append([counts, segment, classificationNumber]) #counts, segmentID, classification number
            self.countList = countList
            
        self.parent.data.findImgId()

        croppedImage = self.parent.imageDisplay.imageData
        maskedImage = self.parent.imageDisplay.maskImage(croppedImage)
        maskSavePath = os.path.relpath(self.parent.data.maskDir + './SegmentMask-' + str(self.parent.data.imgID) + '.jpg')

        plt.imsave(maskSavePath, maskedImage)
            
        if self.parent.data.COCOFileName == None:
            self.parent.data.createCOCOFileName(saveType)
            
        print(self.parent.data.COCOFileName)
        self.parent.lButtons.tdsLabel.config(text='Current File Name:\n' + str(self.parent.data.COCOFileName))
            
        # Change training datatset to reflect changes
        self.parent.data.CC.toCOCO(self.parent.data.COCOFileName, trainingData=self.parent.event.countList, imgId=self.parent.data.imgID, imgName=self.parent.data.imageName,
                                   imgHeight=self.parent.imageDisplay.imageZoom * 2, imgWidth=self.parent.imageDisplay.imageZoom * 2)
        
            
    # Function to classify the segments
    def classify(self, keyPress):
        if not self.isActive:
            return

        # Assigning the highlighted segment a classification
        segmentIDs = [self.parent.data.segmentList[i] for i in self.parent.data.segNumber]
        
        equalsSegment = list(label[1] for label in self.parent.data.labelVector)
        
        try:
            for i in segmentIDs:
                classified = equalsSegment.index(i)
                del self.parent.data.labelVector[classified]
        except ValueError:
            pass
        
        for segmentID in segmentIDs:
            if keyPress == "water":
                self.parent.data.appendLabel(0, segmentID)
            elif keyPress == "thinIce":
                self.parent.data.appendLabel(1, segmentID)
            elif keyPress == "shadow":
                self.parent.data.appendLabel(2, segmentID)
            elif keyPress == "subIce":
                self.parent.data.appendLabel(3, segmentID)
            elif keyPress == "snow":
                self.parent.data.appendLabel(4, segmentID)
            elif keyPress == "meltPond":
                self.parent.data.appendLabel(5, segmentID)

            # Printing some useful statistics
            print("Assigned value: {} ({})".format(str(self.parent.data.labelVector[-1][0]), keyPress))
            print("To Segment #"+str(segmentID)+" of image "+'"'+self.parent.data.imageName+'"')
            print("At: ("+str(self.parent.imageDisplay.xMin)+", "+str(self.parent.imageDisplay.yMin) + ")")

            #if self.parent.live_predictor.isActive():
             #   self.parent.live_predictor.print_prediction(feature_array)

            print(("~"*80))

        #self.parent.progress_bar.update_progress()
        self.nextSegment()
    

    # Function to cycle to the next image
    def nextImage(self):
        #if not self.isActive:
            #`return 
        self.deactivate()
        # Save data
        self.save() #!TODO: Fix self.save()
        # Set display to loading screen
        self.parent.after(10, self.parent.imageDisplay.loadingScreen())
        # Load next image data
        self.parent.data.loadNextImage()
        # move new data to display
        self.parent.imageDisplay.initializeImage()
        # Remove unlabeled segments from segment list
        # Updates display, moves to next segment, adds segment to queue
        #!TODO self.parent.loadingBar.updateProgress()
        self.activate()
        
        # Navigation Slider Configuration Conditionals
        if self.i < len(self.parent.data.availableImages): # index is atleast n-1 of image array of size n
            self.i += 1 # increment the list for next image config
        if self.i >= len(self.parent.data.availableImages): # index is last image or has exceeded array
            self.i = 0 # reset config index to the begining of the image list
        
        self.parent.imageDisplay.updateImages(0, 0)
        self.parent.data.segNumber = [-1]
        self.parent.data.resetSegments()
        self.configNavSliders() # inform the mainloop to update the nav sliders
        
    # Function to cycle to the previous image
    def prevImage(self):
        self.deactivate()
        # Display to loading screen
        self.parent.after(10, self.parent.imageDisplay.loadingScreen())
        # load previous image data
        self.parent.data.loadPreviousImage()
        # add new data to display class
        self.parent.imageDisplay.initializeImage()
        # Updates display, moves to next segment, adds segment to queue
        #!TODO self.parent.loadingBar.updateProgress()
        self.activate()
        self.parent.imageDisplay.updateImages(0, 0)
        self.parent.data.segNumber = [-1]
        self.parent.data.resetSegments()
        
        # Navigation Slider Configuration Conditionals
        if self.gate == False:         # If the gate has been broken
            if self.i > 0: # if this is not the first image in the list
                self.i -= 1 # increment the list for last image config (forward if init_image is pressed twice)
            else: # If the index is zero and the user causes prevImage() to be called we will...
                self.i = len(self.parent.data.availableImages) # change the index of the list to the end of the image list
                                                                # allowing us to traverse the image array from both ends.
        if self.gate: # If the gate has yet to be broken
            self.i = 0
            self.gate = False # Break the one time gate
        
        
        self.configNavSliders() # inform the mainloop to update the nav sliders
        
    # Initializes the image when user presses 'initialize image' button
    def initializeImage(self):
        if len(self.parent.data.availableImages) == 0:
            print("No images to load!")
            return
        # Make sure no image has been loaded already
        if self.parent.data.imageName is not None:
            return
        # prevImage() loads the first image as well
        if self.gate == False: # If uses presses init button a second time     
            self.i += 2 # we index 1 spot over (forward 2 back one) 
        self.prevImage()
        
        
    # Quits the gui and calls save() to save all current data
    def quitEvent(self):
        #Exits gui and automatically saves progress
        self.save()
        self.parent.exitGui()
        
    # This function sends a signal to the main loop that informs it to change the nav slider values
    def sliderBeacon(self):  
        self.beacon = True  
        
    # Funtion that allows navSliders to function properly    
    def configNavSliders(self, zoom = 0):  
        trainingIndex = self.i # we will make them the same...
        imgHeight = self.parent.imageDisplay.imageHeight # this will allow us to retrieve the corresponding image size metadata...
        imgWidth = self.parent.imageDisplay.imageWidth # that is necessary in augmenting the sizes of the nav sliders.
        self.parent.lButtons.yNavSlider.configure(to_ = imgHeight - zoom * 2)# augment nav sliders
        self.parent.lButtons.xNavSlider.configure(to_ = imgWidth - zoom * 2)# !TODO: fix to be more accurate

In [9]:
def main():
    root = tk.Tk()
    inputDir = os.path.relpath('./ReadGUI')  # Find input folder
    try:
        imgList = scrapeDir(inputDir)
    except Exception as e:
        imgList = []
        errorDisplay("No Input Directory", root)
    imgType = 'pan' #!TODO: fix this to be variable - argparser
    
    rasterList = []
    i = 0 # indexing var for creating list of unique raster data for each image
    while i < len(imgList):
        srcDS2 = gdal.Open(imgList[i], gdal.GA_ReadOnly)
        rasterList.append(srcDS2)
        i += 1
        
    #Create root
    #root.geometry('800x800')
    #root.attributes('-fullscreen', True) #!TODO VVV activate_autorun = autorun_flag insert into TrainingWindow args
      
    #Initialize Gui
    window = TrainingWindow(root, imgList, imgType, rasterList)
    window.pack(fill='both', expand=True)

    root.resizable(False, False)
    root.protocol("WM_DELETE_WINDOW", window.exitGui)
    root.mainloop()

In [10]:
main()